In [1]:
#Import required libraries
!pip install pandas
!pip install folium
import pandas as pd
import folium
import requests
import json

     |████████████████████████████████| 9.9 MB 19.1 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 83.8 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 3.4 MB/s  eta 0:00:01


In [2]:
#Import San Francisco Crime data
df_incidents = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Police_Department_Incidents_-_Previous_Year__2016_.csv')

print('Dataset downloaded and read into a pandas dataframe!'
     ,'Shape= ',df_incidents.shape)

Dataset downloaded and read into a pandas dataframe! Shape=  (150500, 13)


In [3]:
#Check latest date of incident data in order to call Foursquare API for correct time point
max_date=df_incidents['Date'].max()
max_date=max_date[0:10]
API_date=max_date[6:10]+max_date[0:2]+max_date[3:5]
API_date

'20161231'

In [4]:
#Data wrangling - only keep criminal incidents & drop irrelevant columns
df_incidents.rename(columns={'Y':'Latitude','X':'Longitude'},inplace=True)
df_incidents.drop(axis=1,columns=['IncidntNum','DayOfWeek','Date','Time','PdId','Location'],inplace=True)
df_incidents= df_incidents[df_incidents.Category != 'NON-CRIMINAL']
print(df_incidents.shape)
df_incidents.head()

(132634, 7)


,Category,Descript,PdDistrict,Resolution,Address,Longitude,Latitude
0,WEAPON LAWS,POSS OF PROHIBITED WEAPON,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421
1,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421
2,WARRANTS,WARRANT ARREST,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981
5,ASSAULT,BATTERY,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019
6,OTHER OFFENSES,PAROLE VIOLATION,SOUTHERN,"ARREST, BOOKED",MARY ST / HOWARD ST,-122.405721,37.780879


In [137]:
sf_lat = 37.7649
sf_long = -122.4194
map_sf=folium.Map(location=[sf_lat,sf_long],zoom_start=12)
map_sf

In [70]:
CLIENT_ID = 'H41B2EZTCCABXAN4W2VQMMEXVWOF2GYP2MZ3QLLRVSLIQSQF' # your Foursquare ID
CLIENT_SECRET = 'YOZHRR2KQROR0JT5PSSGPMNHIRSQDU2HEYB5UWUOPP1ODRTE' # your Foursquare Secret
VERSION = API_date # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


# create the API request URL
API_url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    sf_lat, 
    sf_long, 
    'policestation')
            
    
print('API_url: ',API_url)

API_url:  https://api.foursquare.com/v2/venues/search?&client_id=H41B2EZTCCABXAN4W2VQMMEXVWOF2GYP2MZ3QLLRVSLIQSQF&client_secret=YOZHRR2KQROR0JT5PSSGPMNHIRSQDU2HEYB5UWUOPP1ODRTE&v=20161231&ll=37.7449,-122.4194&query=policestation


In [72]:
# make the GET request
results = requests.get(API_url).json()["response"]["venues"]
results

[{'id': '4b43f6acf964a52028ef25e3',
  'name': 'Tenderloin Police Station',
  'contact': {},
  'location': {'address': '301 Eddy St',
   'crossStreet': 'at Jones',
   'lat': 37.78382621650408,
   'lng': -122.41293091990042,
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.78382621650408,
     'lng': -122.41293091990042}],
   'distance': 4370,
   'postalCode': '94102',
   'cc': 'US',
   'neighborhood': 'Tenderloin',
   'city': 'San Francisco',
   'state': 'CA',
   'country': 'United States',
   'formattedAddress': ['301 Eddy St (at Jones)',
    'San Francisco, CA 94102',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d12e941735',
    'name': 'Police Station',
    'pluralName': 'Police Stations',
    'shortName': 'Police Station',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_policestation_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,


In [73]:
station_list=[]

for item in results:
    my_dict={}
    my_dict['name']=item.get('name')
    my_dict['latitude']=item.get('location').get('lat')
    my_dict['longitude']=item.get('location').get('lng')
    my_dict['address']=item.get('location').get('address')
    my_dict['neighbourhood']=item.get('location').get('neighborhood')
    my_dict['city']=item.get('location').get('city')
    my_dict['id']=item.get('id')
    station_list.append(my_dict)

station_list

[{'name': 'Tenderloin Police Station',
  'latitude': 37.78382621650408,
  'longitude': -122.41293091990042,
  'address': '301 Eddy St',
  'neighbourhood': 'Tenderloin',
  'city': 'San Francisco',
  'id': '4b43f6acf964a52028ef25e3'},
 {'name': 'UCSF Police Station Mission Bay',
  'latitude': 37.7674674987793,
  'longitude': -122.39086151123047,
  'address': '600 16th St, Genentech Hall 124',
  'neighbourhood': None,
  'city': 'San Francisco',
  'id': '4e85d2195c5cfedb827a5451'},
 {'name': 'Police Station - Fillmore And Turk',
  'latitude': 37.78041064739227,
  'longitude': -122.4321985244751,
  'address': '1401-1499 Turk St',
  'neighbourhood': None,
  'city': 'San Francisco',
  'id': '4d9cf839a1ec8cfad185f849'},
 {'name': 'Police Station',
  'latitude': 37.77714538574219,
  'longitude': -122.40219116210938,
  'address': '70 Oak Grove St',
  'neighbourhood': None,
  'city': 'San Francisco',
  'id': '43582f80f964a520db281fe3'},
 {'name': 'Central District Police Station',
  'latitude': 3

In [135]:
from pandas import DataFrame

results=DataFrame(station_list)
results.drop(results[results['city']!='San Francisco'].index,inplace=True)
results.drop(results[results['name']=='Richmond Police Station Community Room'].index,inplace=True)
results.reset_index(drop=True,inplace=True)
results

,name,latitude,longitude,address,neighbourhood,city,id
0,Tenderloin Police Station,37.783826,-122.412931,301 Eddy St,Tenderloin,San Francisco,4b43f6acf964a52028ef25e3
1,UCSF Police Station Mission Bay,37.767467,-122.390862,"600 16th St, Genentech Hall 124",None,San Francisco,4e85d2195c5cfedb827a5451
2,Police Station - Fillmore And Turk,37.780411,-122.432199,1401-1499 Turk St,None,San Francisco,4d9cf839a1ec8cfad185f849
3,Police Station,37.777145,-122.402191,70 Oak Grove St,None,San Francisco,43582f80f964a520db281fe3
4,Central District Police Station,37.798528,-122.409704,766 Vallejo St,None,San Francisco,4ac232baf964a5202a9820e3
5,Potrero Hill Police Station,37.751912,-122.400039,401-495 De Haro St,None,San Francisco,4aaa7906f964a520155620e3
6,Mission Police Station,37.762975,-122.421723,630 Valencia St,None,San Francisco,4af0f30af964a52047e021e3
7,Old Mission Police Station,37.763842,-122.414216,None,None,San Francisco,4c4b55dc46240f4769fc2ef3
8,Bayview Police Station,37.729880,-122.397979,201 Williams Ave,None,San Francisco,4c50c1d6991c20a10c39a986
9,Park Police Station,37.768003,-122.455309,1899 Waller St,None,San Francisco,4aecc91df964a5202ecb21e3


In [138]:
for lat,long,label in zip(results['latitude'],results['longitude'],results['name']):
    folium.CircleMarker(location=[lat,long],radius=5,color='darkblue',fill=True,fill_color='darkblue',fill_opacity=1,popup=label).add_to(map_sf)
map_sf

In [120]:
df_incidents.groupby('Category').count().sort_values(by='Descript',ascending=False)

,Descript,PdDistrict,Resolution,Address,Longitude,Latitude
Category,,,,,,
LARCENY/THEFT,40409,40408,40409,40409,40409,40409
OTHER OFFENSES,19599,19599,19599,19599,19599,19599
ASSAULT,13577,13577,13577,13577,13577,13577
VANDALISM,8589,8589,8589,8589,8589,8589
VEHICLE THEFT,6419,6419,6419,6419,6419,6419
WARRANTS,5914,5914,5914,5914,5914,5914
BURGLARY,5802,5802,5802,5802,5802,5802
SUSPICIOUS OCC,5782,5782,5782,5782,5782,5782
MISSING PERSON,4338,4338,4338,4338,4338,4338


In [121]:
#to drop:
 #   warrants
 #   suicide
 #   other offenses
 #   Trea
    
#Combine:
 #   Larceny/theft & robbery & stolen property

df_incidents_sub=df_incidents[df_incidents['Category'].isin(['WARRANTS','SUICDE','OTHER OFFENSES','TREA']) == False].reset_index(drop=True)
print(df_incidents.shape)
df_incidents_sub.head(10)


(132634, 7)


,Category,Descript,PdDistrict,Resolution,Address,Longitude,Latitude
0,WEAPON LAWS,POSS OF PROHIBITED WEAPON,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421
1,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421
2,ASSAULT,BATTERY,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019
3,MISSING PERSON,FOUND PERSON,BAYVIEW,NONE,100 Block of CAMERON WY,-122.387182,37.720967
4,LARCENY/THEFT,ATTEMPTED THEFT FROM LOCKED VEHICLE,TARAVAL,NONE,1200 Block of 19TH AV,-122.477377,37.764478
5,ASSAULT,AGGRAVATED ASSAULT WITH A GUN,INGLESIDE,"ARREST, BOOKED",SILVER AV / CAMELLIA AV,-122.432326,37.729271
6,BURGLARY,"BURGLARY,STORE UNDER CONSTRUCTION, FORCIBLE ENTRY",CENTRAL,NONE,100 Block of SANSOME ST,-122.400909,37.791643
7,STOLEN PROPERTY,"STOLEN CHECKS, POSSESSION",SOUTHERN,"ARREST, BOOKED",900 Block of MARKET ST,-122.408595,37.783707
8,ROBBERY,"ROBBERY, ARMED WITH A KNIFE",MISSION,NONE,900 Block of POTRERO AV,-122.406870,37.757290
9,ASSAULT,BATTERY WITH SERIOUS INJURIES,MISSION,NONE,VALENCIA ST / 26TH ST,-122.420355,37.748906
